In [6]:
import numpy as np
from transformers import BertTokenizer, AutoModel
import torch
import datasets
import os
import argparse
from model.gpt2 import GPT2Model, GPT2Config

In [7]:
our_model = GPT2Model(GPT2Config(hidden_size=768, num_hidden_layers=12,num_attention_heads=12,
                                         intermediate_size=768*3, use_lora = True)).eval()

In [8]:
for name, parameter in our_model.named_parameters():
    print(name)

word_embedding.weight
pos_embedding.weight
gpt_layers.0.self_attention.query.lora_A
gpt_layers.0.self_attention.query.lora_B
gpt_layers.0.self_attention.query.linear.weight
gpt_layers.0.self_attention.query.linear.bias
gpt_layers.0.self_attention.key.lora_A
gpt_layers.0.self_attention.key.lora_B
gpt_layers.0.self_attention.key.linear.weight
gpt_layers.0.self_attention.key.linear.bias
gpt_layers.0.self_attention.value.lora_A
gpt_layers.0.self_attention.value.lora_B
gpt_layers.0.self_attention.value.linear.weight
gpt_layers.0.self_attention.value.linear.bias
gpt_layers.0.attention_dense.lora_A
gpt_layers.0.attention_dense.lora_B
gpt_layers.0.attention_dense.linear.weight
gpt_layers.0.attention_dense.linear.bias
gpt_layers.0.attention_layer_norm.weight
gpt_layers.0.attention_layer_norm.bias
gpt_layers.0.interm_dense.lora_A
gpt_layers.0.interm_dense.lora_B
gpt_layers.0.interm_dense.linear.weight
gpt_layers.0.interm_dense.linear.bias
gpt_layers.0.out_dense.lora_A
gpt_layers.0.out_dense.lora

In [ ]:
torch.zeros(3, 5)

In [ ]:
for name in our_model.named_children():
    print(name)

In [4]:
# Test the LoRA functionality to diagnose the issue
from modules.lora_linear import LoRALinear

# Create test tensors with shapes from our error case
in_features = 768
out_features = 2304
batch_size = 8
seq_len = 86

# Test LoRALinear with use_lora=True
lora_layer = LoRALinear(in_features, out_features, r=4, use_lora=True)
input_tensor = torch.randn(batch_size, seq_len, in_features)

# Check shapes before forward pass
print(f"Input tensor shape: {input_tensor.shape}")
print(f"Linear weight shape: {lora_layer.linear.weight.shape}")
print(f"LoRA A shape: {lora_layer.lora_A.shape}")
print(f"LoRA B shape: {lora_layer.lora_B.shape}")

# Run forward pass
output = lora_layer(input_tensor)
print(f"Output shape: {output.shape}")

# Verify shapes manually
linear_output = lora_layer.linear(input_tensor)
print(f"Linear output shape: {linear_output.shape}")

lora_A_proj = torch.matmul(input_tensor, lora_layer.lora_A.T)
print(f"lora_A_proj shape: {lora_A_proj.shape}")

lora_output = torch.matmul(lora_A_proj, lora_layer.lora_B.T) * lora_layer.scaling
print(f"lora_output shape: {lora_output.shape}")

Input tensor shape: torch.Size([8, 86, 768])
Linear weight shape: torch.Size([2304, 768])
LoRA A shape: torch.Size([4, 768])
LoRA B shape: torch.Size([2304, 4])
Output shape: torch.Size([8, 86, 2304])
Linear output shape: torch.Size([8, 86, 2304])
lora_A_proj shape: torch.Size([8, 86, 4])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (688x4 and 768x4)